In [ ]:
import sys, random
sys.path.append('../reconstruction')
sys.path.append('../utils')

from reconstruct_atom_data import *
from pyatom import Atmosphere, Hydrosphere

#create the models
atm_model = Atmosphere()
hyd_model = Hydrosphere()

#load configurations
atm_model.load_config( './config_atm.xml' )
hyd_model.load_config( './config_hyd.xml' )

#get start time, end time and time step from configurations
start_time = 0
end_time = 6
time_step = 5

times = range(start_time, end_time+1, time_step)

#run the models
BATHYMETRY_SUFFIX = 'Ma_smooth.xyz'
for idx, time in enumerate(times):
    atm_model.run_time_slice(time)
    hyd_model.run_time_slice(time)
    if idx<len(times)-1:
        reconstruct_temperature(time,times[idx+1], BATHYMETRY_SUFFIX) 
        reconstruct_precipitation(time,times[idx+1], BATHYMETRY_SUFFIX)
        reconstruct_salinity(time,times[idx+1], BATHYMETRY_SUFFIX)



In [ ]:
import sys
sys.path.append('../reconstruction')
sys.path.append('../utils')
import create_atm_maps, create_hyd_maps

start_time = 0
end_time = 6
time_step = 5

import create_atm_maps, create_hyd_maps

try:
    topo_dir = '../data/topo_grids/'
    topo_suffix = 'smooth'
    atm_map_output_dir = './atm_maps'
    hyd_map_output_dir = './hyd_maps'

    # v-velocity(m/s), w-velocity(m/s), velocity-mag(m/s), temperature(Celsius), water_vapour(g/kg), 
    # precipitation(mm), precipitable water(mm)
    atm_sub_dirs = ['temperature','v_velocity','w_velocity', 'water_vapour', 'precipitation', 
                'precipitable_water', 'topography', 'velocity']

    create_atm_maps.create_all_maps(atm_sub_dirs, start_time, end_time, time_step, atm_map_output_dir, 
            './output', topo_dir, topo_suffix)

    hyd_sub_dirs = ['temperature','v_velocity','w_velocity', 'salinity', 'bottom_water', 
            'upwelling', 'downwelling', 'velocity']
    
    create_hyd_maps.create_all_maps(hyd_sub_dirs, start_time, end_time, time_step, hyd_map_output_dir,
            './output', topo_dir, topo_suffix)
except:
    import traceback
    traceback.print_exc() 


In [ ]:
data=[]
for lat in range(-90,90):
    data.append(atm_model.get_temperature(5,220,10,lat))
    
import matplotlib.pyplot as plt
plt.plot(data)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np

data=[]

for lat in range(-90,91):
    tmp=[]
    for lon in range(-180,181):
        tmp.append(atm_model.get_temperature(0,20,lon,lat))
    data.append(tmp)
    

plt.figure(figsize=(15, 8))

m = Basemap(llcrnrlon=-180,llcrnrlat=-90,urcrnrlon=180,urcrnrlat=90,projection='kav7', lon_0=0)

z=np.array(data)
zz = z.reshape((181, 361))
#zz = np.flipud(zz)

img_data = m.transform_scalar(zz, np.arange(-180,180),np.arange(-90,90),361,181)
cs = m.imshow(img_data,alpha=0.5, vmin=-60, vmax=40, cmap='jet')


m.drawparallels(np.arange(-90., 90., 10.), labels=[1,0,0,0], fontsize=10)
m.drawmeridians(np.arange(-180., 180., 45.), labels=[0,0,0,1], fontsize=10)

plt.show()